# Iterative Methods #
    - Matt Robinson

## Jacobi's Method:

We start with our square system of $n$ linear equations

$$
\mathbf{A}\mathbf{x} = \mathbf{b}
$$

We then decompose $\mathbf{A}$ into the sum of strictly lower triangular matrix $\mathbf{L}$, a strictly diagonal matrix $\mathbf{D}$, and a strictly upper triangular matrix $\mathbf{U}$:

$$
\mathbf{A} = \mathbf{L} + \mathbf{D} + \mathbf{U}
$$

Using this decomposition, we can now write our system as follows:

$$
(\mathbf{L} + \mathbf{D} + \mathbf{U})\mathbf{x} = \mathbf{b} \\
\implies \mathbf{D}\mathbf{x} = -(\mathbf{L} + \mathbf{U})\mathbf{x} + \mathbf{b}\\
$$

We then solve for the solution through an iterative scheme:

$$
\mathbf{D}\mathbf{x}^{(k+1)} = -(\mathbf{L} + \mathbf{U})\mathbf{x}^{(k)} + \mathbf{b}\\
\implies \mathbf{x}^{(k+1)} = \mathbf{D}^{-1} \left(\mathbf{b} -(\mathbf{L} + \mathbf{U})\mathbf{x}^{(k)}\right)\\
$$

Note that this shows that the iteration matrix $\mathbf{H}_J = \mathbf{D}^{-1}(\mathbf{L} + \mathbf{U})$. Futhermore, the above formula can be expressed elementwise to get:

$$
x _ { i } ^ { ( k + 1 ) } = \frac { 1 } { A _ { i i } } \left( b _ { i } - \sum _ { j \neq i } A _ { i j } x _ { j } ^ { ( k ) } \right) , \quad i = 1,2 , \ldots , n
$$

Many books will justify this formula more simply by starting with the elementwise form of $\mathbf{A}\mathbf{x} = \mathbf{b}$ and separating out the diagonal components. For example, see the following argument copied from Professor Gerard Gorman's [numerical analysis notes](https://nbviewer.jupyter.org/github/ggorman/Numerical-methods-1/blob/master/notebook/Lecture-7-Numerical-methods-1.ipynb)


# ""
### ggorman's element-wise explanation:
Copied from link above.

Consider our matrix system

$$A\pmb{x}=\pmb{b} \quad \iff \quad \sum_{j=1}^nA_{ij}x_j=b_i,\quad \textrm{for}\quad i=1,2,\ldots, n.$$

Let's rewrite this by pulling out the term involving $x_i$ (i.e. for each row $i$ pull out the diagonal from the summation):

$$A_{ii}x_i + \sum_{\substack{j=1\\ j\ne i}}^nA_{ij}x_j=b_i,\quad  i=1,2,\ldots, n.$$

We can then come up with a formula for our unknown $x_i$:

$$x_i = \frac{1}{A_{ii}}\left(b_i- \sum_{\substack{j=1\\ j\ne i}}^nA_{ij}x_j\right),\quad  i=1,2,\ldots, n.$$

Now of course for each individual $x_i$, all the other components of $\pmb{x}$ appearing on the RHS are also unknown and so this is an example of an implicit formula which doesn't help us directly, but does suggest the following iterative scheme:

* Starting from a guess at the solution $\pmb{x}^{(0)}$

* iterate for $k>0$
$$x_i^{(k+1)} = \frac{1}{A_{ii}}\left(b_i- \sum_{\substack{j=1\\ j\ne i}}^nA_{ij}x_j^{(k)}\right),\quad  i=1,2,\ldots, n.$$

Note that for this iteration, for a fixed $k$, it does not matter in which order we perform the operations over $i$ as the right hand side only contains the entries of $\pmb{x}$ at the previous iteration.

### end of ggorman's explanation
# ""



In [1]:
import numpy as np

In [3]:
def Jacobi(A, b, tol=1.0E-6, max_iter=100):
    
    # initial guess at the solution - just use zero vector for now
    x = np.zeros(A.shape[1])
    # inital x^{k+1} vector
    x_new = np.empty(A.shape[1])
    
    for _ in range(max_iter):
        
        D = np.diag(np.diag(A)) 
        L_plus_U = A - D
        
        x_new = np.linalg.inv(D).dot(b - L_plus_U.dot(x))
        
        error = np.linalg.norm(A.dot(x_new) - b)
        if error < tol:
            return x_new
        else:
            x = x_new
    
    raise RuntimeError('no root found in allowed number of iterations')

In [7]:
A= np.array([[10., 2., 3., 5.],
             [1., 14., 6., 2.],
             [-1., 4., 16., -4],
             [5. ,4. ,3. ,11. ]])

b = np.array([1., 2., 3., 4.])

print("Our Jacobi Method: ", Jacobi(A,b))
print("Numpy: ", np.dot(np.linalg.inv(A),b))

Our Jacobi Method:  [-0.16340807 -0.01532701  0.27335259  0.36893548]
Numpy:  [-0.16340816 -0.01532706  0.27335264  0.36893555]


## The Gauss-Seidel Method ##

The element-wise formulation of Jacobi's method shows that we only use the components of the last iteration $x _ { i } ^ { ( k ) }$ in constructing $x _ { i } ^ { ( k + 1 ) }$. Naturally, one wonders what would happen if we use the updated components of the solution vector $x _ { i } ^ { ( k + 1 ) }$ as soon they become available. That is, a component $x _ { j } ^ { ( k + 1 ) }$ might rely on $x _ { i } ^ { ( k + 1 ) }$ instead of $x _ { i } ^ { ( k ) }$. This improvement to the Jacobi method is known as the *Gauss-Seidel* Method, which is often faster than Jacobi.

$$
x _ { i } ^ { ( k+1 ) } = \frac { 1 } { A _ { i i } } \left( b _ { i } - \sum _ { j = 1 \atop j<i} ^ { n } A _ { i j } x _ { j } ^ { ( k+1 ) } - \sum _ { j = 1 \atop j > i } ^ { n } A _ { i j } x _ { j } ^ { ( k ) } \right) , \quad i = 1,2 , \ldots , n
$$

Let's now look at the matrix formulation of the Gauss-Seidel method. We start again by decomposing $\mathbf{A}$:

$$
(\mathbf{L} + \mathbf{D} + \mathbf{U})\mathbf{x} = \mathbf{b} \\
\implies (\mathbf{L} + \mathbf{D})\mathbf{x}^{(k+1)} = - \mathbf{U}\mathbf{x}^{(k)} + \mathbf{b}\\
$$

We see that our iteration matrix could then be calculated as $\mathbf{H} _ { G S } = - ( \mathbf{L} + \mathbf{D} ) ^ { - 1 } \mathbf{U}$. However, in practice, there is no need for us to invert $( \mathbf{L} + \mathbf{D} )$. Since $( \mathbf{L} + \mathbf{D} )$ is a strictly lower triangular matrix (does not include the diagonal) plus a diagonal matrix, we can simply use forward substitution to solve this system. Using forward substitution leads to the exact same element-wise formula as above.

In [18]:
def GS(A, b, tol=1.0E-6, max_iter=100):
    
    # initial guess at the solution - just use zero vector for now
    x = np.zeros(A.shape[1])
    # inital x^{k+1} vector
    x_new = np.empty(A.shape[1])
    
    for _ in range(max_iter):
        
        for i in range(len(x_new)):
            x_new[i] = (1/A[i, i]) * (b[i] - np.dot(A[i, :i], x_new[:i]) - np.dot(A[i, i + 1:], x[i + 1:]))
        
        error = np.linalg.norm(A.dot(x_new) - b)
        if error < tol:
            return x_new
        else:
            x = x_new
    
    raise RuntimeError('no root found in allowed number of iterations')

In [19]:
A= np.array([[10., 2., 3., 5.],
             [1., 14., 6., 2.],
             [-1., 4., 16., -4],
             [5. ,4. ,3. ,11. ]])

b = np.array([1., 2., 3., 4.])

print("Our Gauss-Seidel Method: ", GS(A,b))
print("Numpy: ", np.dot(np.linalg.inv(A),b))

Our Gauss-Seidel Method:  [-0.16340812 -0.01532701  0.27335261  0.36893553]
Numpy:  [-0.16340816 -0.01532706  0.27335264  0.36893555]


## When do these methods work?

In [12]:
# a quick counter example
A= np.array([[1, 2],
             [2, 1]])

b = np.array([5, 6])
print("Numpy: ", np.dot(np.linalg.inv(A),b))

Numpy:  [2.33333333 1.33333333]


In [16]:
print("Our Jacobi Method: ", Jacobi(A,b,max_iter=1000))

RuntimeError: no root found in allowed number of iterations

In [17]:
print("Our Gauss-Seidel Method: ", GS(A,b,max_iter=1000))

RuntimeError: no root found in allowed number of iterations

Obviously these methods don't work for every system. Let's go through some conditions for their success.

**FACT**: The sequence converges to the solution if the spectral radius (max absolute value of eigenvalues) of the iteration matrix is less than one. For Jacobi, the iteration matrix is $\mathbf{H}_J = \mathbf{D}^{-1}(\mathbf{L} + \mathbf{U})$, while for Gauss-Seidel the iteration matrix is $\mathbf{H} _ { G S } = - ( \mathbf{L} + \mathbf{D} ) ^ { - 1 } \mathbf{U}$. 

One can prove that the spectral radius of these matrices is less than one for special classes of matrices, and thus we can discover under which classes of matrices our methods will converge.

**Theorem**: If $\mathbf{A}$ is strictly *diagonally dominant*, then both Jacobi and Gauss-Seidel will converge. A matrix is defined as strictly diagonally dominant if, for every row, the magnitude of the diagonal element is greater than the sum of all of the other elements in the row. That is,

$$
\left| A _ { i , i } \right| > \sum _ { j = 1 \atop j \neq i } ^ { n } \left| A _ { i , j } \right| \quad \text{for all } i
$$

**Theorem**: If $\mathbf{A}$ is symmetric and positive definite, then the Gauss-Seidel method will converge.

Note that this does not necessarily guarentee the convergence of Jacobi. In this case, $\mathbf{A}$ and $2\mathbf{D} -\mathbf{A}$ must both be symmetric and positive definite to guarentee convergence.

In [22]:
# from ggorman
def is_diag_dom(A):
        
    (n,m) = A.shape
    if n != m: 
        print("Error: matrix must be square, not %dx%d" % (n,m))
        exit(1)
    
    for i in range(n):
        abs_row = np.sum(np.abs(A[i,0:i])) + np.sum(np.abs(A[i,i+1:n]))
        if abs_row > abs(A[i,i]):
            print("!! Row %d is not diagonally dominant" %i)
            return False
        
    return True

In [20]:
# Below I am writing Gauss-Seidel with relaxation

def relaxed_GS(A, b, omega=1, tol=1.0E-6, max_iter=1000):
    
    # initial guess at the solution - just use zero vector for now
    x = np.zeros(A.shape[1])
    # inital x^{k+1} vector
    x_new = np.empty(A.shape[1])
    
    for iteration in range(max_iter):
        
        for i in range(len(x_new)):
            x_new[i] = ( (omega/A[i, i]) * (b[i] - np.dot(A[i, :i], x_new[:i]) - np.dot(A[i, i + 1:], x[i + 1:])) 
                        + (1-omega)*x[i] )
        
        error = np.linalg.norm(A.dot(x_new) - b)
        if error < tol:
            return x_new, iteration
        else:
            x = x_new
    
    raise RuntimeError('no root found in allowed number of iterations')

In [24]:
A = np.array([[10., 2., 3., 5.],
                [1., 14., 6., 2.],
                 [-1., 4., 16.,-4],
                 [5. ,4. ,3. ,11.]])
b = np.array([1., 2., 3., 4.])

print(is_diag_dom(A))

!! Row 3 is not diagonally dominant
False


In [25]:
for omega in [0.5, 0.9, 0.95, 1, 1.1, 1.5]:
    x,i = relaxed_GS(A, b, omega)
    print("For omega=%1.2f  %d relaxed GS iterations are needed to reach a tolerance of 1e-6" % (omega,i))

For omega=0.50  48 relaxed GS iterations are needed to reach a tolerance of 1e-6
For omega=0.90  18 relaxed GS iterations are needed to reach a tolerance of 1e-6
For omega=0.95  16 relaxed GS iterations are needed to reach a tolerance of 1e-6
For omega=1.00  14 relaxed GS iterations are needed to reach a tolerance of 1e-6
For omega=1.10  12 relaxed GS iterations are needed to reach a tolerance of 1e-6
For omega=1.50  29 relaxed GS iterations are needed to reach a tolerance of 1e-6


# Steepest Descent and Conjugate Gradient Methods #

Here I will mostly just supply the code, see Faul's notes for more details.

Note that $\mathbf{A}$ must be symmetric and positive definite for the method to work.

In [27]:
def SD(A, b, tol=1.0E-6, max_iter=1000):
    
    if not np.array_equal(A,A.T):
        raise ValueError('A is not symmetric')
    
    for e_val in np.linalg.eigvals(A):
        if e_val <= 0:
            raise ValueError('A is not positive definite') 
    
    # initial guess at the solution - just use zero vector for now
    x = np.zeros(A.shape[1])
    # inital x^{k+1} vector
    x_new = np.empty(A.shape[1])
    
    for iteration in range(max_iter):
        
        # calculate gradient, search direction, and step size
        grad = A.dot(x) - b
        search_direction = -1*grad
        omega = (grad.dot(grad))/(grad.dot(A.dot(grad)))
        
        x_new = x + omega*search_direction
        
        gradient_norm = np.linalg.norm(A.dot(x_new) - b)
        if gradient_norm < tol:
            return x_new, iteration
        else:
            x = x_new
    
    raise RuntimeError('No root found in allowed number of iterations')

In [32]:
A = np.array([[2, -1., 0],
              [-1., 2, -1.],
              [0., -1., 2]])
b = np.array([1., 2., 3.])

x, num_iters = SD(A,b)
print("Our Steepest Descent Method: ", x)
print("Numpy: ", np.dot(np.linalg.inv(A),b))

Our Steepest Descent Method:  [2.49999962 3.99999923 3.49999962]
Numpy:  [2.5 4.  3.5]


Now let's code the conjugate gradient method.

In [5]:
def CG(A, b, tol=1.0E-6, max_iter=8):
    '''
    See http://www.acme.byu.edu/wp-content/uploads/2014/09/Vol2Lab18ConjugateGradient.pdf
    for correct description of algorithm implementation
    '''
    
    if not np.array_equal(A,A.T):
        raise ValueError('A is not symmetric')
    
    for e_val in np.linalg.eigvals(A):
        if e_val <= 0:
            raise ValueError('A is not positive definite') 
    
    # initial guess at the solution - just use 0 vector for now
    x = np.zeros(A.shape[1])
    # intialize the residual = -1*gradient. Initially this is b, since x0 is 0 vector
    r = A.dot(x)-b
    # initialize search direction to -1*gradient = residual
    d = -1*r
    
    # inital {k+1} vectors
    x_new = np.empty(len(x))
    r_new = np.empty(len(r))
    d_new = np.empty(len(d))
    
    for iteration in range(max_iter):
        
        # calculate gradient, search direction, and step size
        omega = (r.dot(r))/(d.dot(A.dot(d)))
        
        x_new = x + omega*d
        r_new = r + omega*(A.dot(d))
        
        # print(x_new)
        
        beta = (r_new.dot(r_new))/(r.dot(r))
        d_new = -r_new + beta*d
        
        if np.linalg.norm(r_new) < tol:
            return x_new, iteration
        else:
            x = x_new
            r = r_new
            d = d_new
    
    raise RuntimeError('No root found in allowed number of iterations')

In [6]:
A = np.array([[2, -1., 0],
              [-1., 2, -1.],
              [0., -1., 2]])
b = np.array([1., 2., 3.])


x, num_iters = CG(A,b)
print("Our Conjugate Gradient Method: ", x)
print("Numer of Iterations: ", num_iters)
print("Numpy: ", np.dot(np.linalg.inv(A),b))

Our Conjugate Gradient Method:  [2.5 4.  3.5]
Numer of Iterations:  2
Numpy:  [2.5 4.  3.5]


In [13]:
def power_method(A, tol=1.0E-3, max_iter=100):
    
    if not np.array_equal(A,A.T):
        raise ValueError('A is not symmetric')
    
    # initial guess - just use ones vector
    # then normalize in order to make it a unit vector
    x = np.ones(A.shape[1])/np.linalg.norm(np.ones(A.shape[1]))
    
    # inital x^{k+1} vector
    x_new = np.empty(A.shape[1])
    
    for iteration in range(max_iter):
        
        x_new = A.dot(x)
        min_lambda = (x.dot(A.dot(x)))/(x.dot(x))
        
        error = np.linalg.norm(x_new - min_lambda*x)
        if error < tol:
            return x_new/np.linalg.norm(x_new), min_lambda, iteration
        else:
            x = x_new/np.linalg.norm(x_new)
    
    raise RuntimeError('no root found in allowed number of iterations')

In [14]:
e_vec, e_val, num_iters = power_method(A)
print("Power Method Eigenvector: ",e_vec)
print("Numpy Eigenvector: ", np.linalg.eig(A)[1][0],"\n")

print("Power Method Eigenvalue: ",e_val)
print("Numpy Eigenvalue: ", np.linalg.eigvals(A)[0])

Power Method Eigenvector:  [ 0.50001275 -0.70708874  0.50001275]
Numpy Eigenvector:  [-0.5        -0.70710678  0.5       ] 

Power Method Eigenvalue:  3.4142134998513236
Numpy Eigenvalue:  3.4142135623730914
